## Random Forest for ICB

In [1]:
import os 
import pandas as pd
import numpy as np
import sklearn

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [2]:
# Implementation Dependant Globals
REQUIRED_DIMENSIONS = (224, 224)
# These affect only the generators, we don't use those for KNN, we use the raw congregrates.
TRAIN_BATCH_SIZE, VALIDATION_BATCH_SIZE, TEST_BATCH_SIZE, N_AUG_VS_TEST = 0, 0, 0, 0

# Import dataset
%run DataPreparation.ipynb 

[14:15:53] Library functions loaded.
[14:15:56] Data preparation started.
[14:15:56] Loaded custom functions.
(SKIPPED) Found a non image file:  ./dataset/validation/non_biomass/metal282 - Shortcut.lnk
[Non Augmented Images]
	 - Number of images in total: 462
	 - Raw pixels matrix: 67.91MB
	 - Raw features matrix: 0.92MB
[Augmented Images]
	 - Number of images in total: 462
	 - Raw pixels matrix: 67.91MB
	 - Raw features matrix: 0.92MB
[14:16:05] Loaded raw congregates of images, features and labels.
[Non Augmented Generators]
Found 248 images belonging to 2 classes.
Found 144 images belonging to 2 classes.
Found 70 images belonging to 2 classes.
[Augmented Generators]
Found 248 images belonging to 2 classes.
Found 70 images belonging to 2 classes.
Found 144 images belonging to 2 classes.
[14:16:06] Loaded all generators.
[14:16:06] Data preperation completed!
Time taken: 10.243054389953613 seconds.


#### Join Congregate Data

In [3]:
# We want to use both augmented dataset and non augmented
# Thus, we join the image data and feature data.
total_rawPixels, total_rawPixelLabels = lib.shuffleJoinRawDatasets(
    rawPixels, Labels, 
    augmented_rawPixels, augmented_Labels
)

#### Parition Congregate Data

In [4]:
# Partition the data into training and testing
# 75% = training, 25% = testing
TEST_SPLIT = 0.3
SEED = 55

(trainImages, testImages, trainImagesLabels, testImagesLabels) = train_test_split(
    total_rawPixels, 
    total_rawPixelLabels, 
    test_size=TEST_SPLIT, 
    random_state=SEED
)

#### Fit Model

In [6]:
model = RandomForestClassifier()

# Fit model
model.fit(trainImages, trainImagesLabels)

C:\ProgramData\Anaconda3\envs\ICB\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Make Predictions on Test Set

In [8]:
# Predictions for test
predictions = model.predict(testImages)

print("Test data metrics:")
print(sklearn.metrics.classification_report(y_true=testImagesLabels, y_pred=predictions))

Test data metrics:
              precision    recall  f1-score   support

     biomass       0.81      0.85      0.83       151
 non_biomass       0.82      0.76      0.79       127

    accuracy                           0.81       278
   macro avg       0.81      0.81      0.81       278
weighted avg       0.81      0.81      0.81       278

